In [6]:
from google.colab import files
uploaded = files.upload()

Saving mscoco_captions_9000.txt to mscoco_captions_9000.txt


In [5]:
from google.colab import files
uploaded = files.upload()

Saving mscoco_captions_1000.txt to mscoco_captions_1000.txt


In [8]:
%matplotlib inline

import matplotlib.pyplot as plt

import numpy as np

data_y = list(open('mscoco_captions_1000.txt'))[:2]
data_x = np.load('mscoco_images_1000.npy')[:2]

plt.figure(figsize=(12, 8))
for i, (x, y) in enumerate(zip(data_x, data_y)):
    plt.subplot(1, 2, i+1)
    plt.imshow(x)
    plt.title(y)
    plt.axis('off')
plt.show()

FileNotFoundError: ignored

In [2]:
from keras.applications.vgg16 import VGG16
from keras.layers import Input

x = Input(shape=(224,224,3))
model = VGG16(
    weights='imagenet', 
    include_top=False,
    input_tensor=x)

Using TensorFlow backend.


58892288/58889256 [==============================] - 5s 0us/step


In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
%matplotlib inline

import os
import sys
import math
import copy

import numpy as np
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Dense, Embedding, LSTM, Activation, Flatten, Reshape, dot, Permute
from keras.optimizers import RMSprop
from keras.utils import np_utils
from keras import backend as K
from keras.layers import Lambda
from sklearn.utils import shuffle

sys.path.append('/root/userspace/public/lesson5/master')
from utils import Node

np.random.seed(34)

emb_dim = 128
hid_dim = 128

In [0]:
def sort_data_by_length(data_x, data_y):
    data_x_lens = [len(datum) for datum in data_x]
    sorted_data_indexes = sorted(range(len(data_x_lens)), key=lambda x: -data_x_lens[x])
    
    data_x = [data_x[i] for i in sorted_data_indexes]
    data_y = [data_y[i] for i in sorted_data_indexes]
    
    return data_x, data_y

def encode(sentence, w2i, unk='<unk>'):
    return [w2i[w] if w in w2i else w2i[unk] for w in sentence]

def build_w2i(path):
    # Build w2i & i2w
    vocab = set()
    for line in open(path):
        sentence = line.strip().split()
        vocab.update(sentence)
    w2i = {w: np.int32(i+4) for i, w in enumerate(vocab)}
    w2i['<s>'], w2i['</s>'], w2i['<unk>'], w2i['<pad>'] = \
        np.int32(0), np.int32(1), np.int32(2), np.int32(3)
    i2w = {i: w for w, i in w2i.items()}
    return w2i, i2w

def build_dataset(path, w2i):
    # Encode data with w2i
    data = []
    for line in open(path):
        sentence = line.strip().split()
        sentence = ['<s>'] + sentence + ['</s>']
        encoded_sentence = encode(sentence, w2i)
        data.append(encoded_sentence)
    
    return data

In [0]:
# ファイルサイズが大きいので少し時間がかかります
train_x = np.load('/root/userspace/public/lesson5/data/mscoco_images_9000.npy')
valid_x = np.load('/root/userspace/public/lesson5/data/mscoco_images_1000.npy')

w2i, i2w = build_w2i('/root/userspace/public/lesson5/data/mscoco_captions_9000.txt')
train_y = build_dataset('/root/userspace/public/lesson5/data/mscoco_captions_9000.txt', w2i)
valid_y = build_dataset('/root/userspace/public/lesson5/data/mscoco_captions_1000.txt', w2i)

vocab_size = len(w2i)

# キャプションの長さでソート (理由は後述)
train_y, train_x = sort_data_by_length(train_y, train_x)
valid_y, valid_x = sort_data_by_length(valid_y, valid_x)

In [0]:
K.clear_session()

x = Input(shape=(224, 224, 3))
encoder = VGG16(weights='imagenet', include_top=False, input_tensor=x)

# Fix parameters on VGG
for layer in encoder.layers:
    layer.trainable = False

u = Flatten()(encoder.output)

In [0]:
# Attention利用時にそなえてreshape: (7, 7, 512) -> (49, 512)
u_map = Reshape((7*7, 512))(u)

# LSTMの初期状態
h_0 = Dense(hid_dim)(u)
cell_0 = Dense(hid_dim)(u)

y = Input(shape=(None,), dtype='int32')
y_in = Lambda(lambda x: x[:, :-1])(y)
y_out = Lambda(lambda x: x[:, 1:])(y)

mask = Lambda(lambda x: K.cast(K.not_equal(x, w2i['<pad>']), 'float32'))(y_out)

embedding = Embedding(vocab_size, emb_dim)
lstm = LSTM(hid_dim, activation='tanh', return_sequences=True, return_state=True)

y_emb = embedding(y_in)
h, _, _ = lstm(y_emb, initial_state=[h_0, cell_0])

In [0]:
# 1. スコアの計算
dense_att = Dense(hid_dim)
_u_map = dense_att(u_map)
score = dot([_u_map, h], axes=-1)

# 2. 重みの計算
permute_att1 = Permute((2, 1))
activation_att = Activation('softmax')
score = permute_att1(score)
a = activation_att(score)

# 3. 文脈ベクトルの計算
permute_att2 = Permute((2, 1))
context = dot([u_map, a], axes=(1, 2))
context = permute_att2(context)

# 4. 出力ベクトルの計算
dense_output1 = Dense(hid_dim)
dense_output2 = Dense(vocab_size)
softmax = Activation('softmax')
h_tilde = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=2))([h, context])
h_tilde = dense_output1(h_tilde)
y_pred = dense_output2(h_tilde)
y_pred = softmax(y_pred)

# マスク化した誤差関数
def cross_entropy(y_true, y_pred):
    return -K.mean(K.sum(K.sum(y_true * K.log(K.clip(y_pred, 1e-10, 1)), axis=-1) * mask, axis=1))

model = Model([x, y], y_pred)

model.compile(loss=cross_entropy, optimizer='rmsprop')

In [0]:
def generator(data_x, data_y, batch_size=32):
    
    n_batches = math.ceil(len(data_x) / batch_size)
    
    while True:
        for i in range(n_batches):
            start = i * batch_size
            end = (i + 1) * batch_size

            data_x_mb = data_x[start:end]
            data_y_mb = data_y[start:end]

            data_x_mb = np.array(data_x_mb).astype('float32') / 255.
            data_y_mb = pad_sequences(data_y_mb, dtype='int32', padding='post', value=w2i['<pad>'])
            data_y_mb_oh = np.array([np_utils.to_categorical(datum_y, vocab_size) for datum_y in data_y_mb[:, 1:]])

            yield [data_x_mb, data_y_mb], data_y_mb_oh

In [0]:
batch_size = 32

n_batches_train = math.ceil(len(train_x) / batch_size)
n_batches_valid = math.ceil(len(valid_x) / batch_size)

try:
    model.fit_generator(
        generator(train_x, train_y),
        epochs=1000,
        steps_per_epoch=n_batches_train,
        validation_data=generator(valid_x, valid_y),
        validation_steps=n_batches_valid
    )
except KeyboardInterrupt:
    pass

In [0]:
encoder_model = Model([x], [u_map, h_0, cell_0])

u_map_inpt = Input(shape=(7*7, 512,))
h_tm1 = Input(shape=(hid_dim,))
cell_tm1 = Input(shape=(hid_dim,))
y_t = Input(shape=(1,))
y_emb_t = embedding(y_t)
_, h_t, cell_t = lstm(y_emb_t, initial_state=[h_tm1, cell_tm1])
h_t = Lambda(lambda x: x[:, None, :])(h_t)
cell_t = Lambda(lambda x: x[:, None, :])(cell_t)

# 1. スコアの計算
_u_map = dense_att(u_map_inpt)
score_t = dot([_u_map, h_t], axes=-1)

# 2. 重みの計算
score_t = permute_att1(score_t)
a_t = activation_att(score_t)

# 3. 文脈ベクトルの計算
context_t = dot([u_map_inpt, a_t], axes=(1, 2))
context_t = permute_att2(context_t)

h_tilde_t = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=2))([h_t, context_t])
h_tilde_t = dense_output1(h_tilde_t)
pred_t = dense_output2(h_tilde_t)

decoder_model = Model([y_t, h_tm1, cell_tm1, u_map_inpt], [pred_t, h_t, cell_t, a_t])

In [0]:
# logの中身が0になるのを防ぐ
def np_log(x):
    return np.log(np.clip(x, 1e-10, x))

In [0]:
def beam_search(x, max_len=100, k=5):
    u_map, h_tm1, cell_tm1 = encoder_model.predict(x)
    y_tm1 = np.array([w2i['<s>']])
    
    root = Node(w2i['<s>'])
    
    # [score, y_tm1, h_tm1, cell_tm1, a, y_pred]
    candidates = [[0, y_tm1, h_tm1, cell_tm1, [], y_tm1]]
    
    t = 0
    while t < max_len:
        root.depth += 1
        t += 1
        
        # すべての候補を一時的に保管するリスト
        tmp_candidates = []
        
        # </s>がすべての候補で出力されたかどうかのフラッグ
        end_flag = True
        for score_tm1, y_tm1, h_tm1, cell_tm1, a, y_pred in candidates:
            a = copy.deepcopy(a)
            if y_tm1[0] == w2i['</s>']:
                tmp_candidates.append([score_tm1, y_tm1, h_tm1, cell_tm1, a, y_pred])
            else:
                end_flag = False
                y_t, h_t, cell_t, a_t = decoder_model.predict([y_tm1, h_tm1, cell_tm1, u_map])
                h_t, cell_t = h_t[:, 0], cell_t[:, 0]
                a.append(a_t.flatten())
                
                # 対数化
                y_t = np_log(y_t.flatten())

                # 確率の高い単語とそのidを取得
                y_t, s_t = np.argsort(y_t)[::-1][:k], np.sort(y_t)[::-1][:k]
                
                # スコア (対数尤度) を蓄積
                score_t = score_tm1 + s_t
                
                # すべての候補を一時的に保管
                tmp_candidates.extend(
                    [[score_tk, np.array([y_tk]), h_t, cell_t, a, np.append(y_pred, [y_tk])]
                        for score_tk, y_tk in zip(score_t, y_t)]
                )
        if end_flag:
            break
        
        # 正規化したスコアでソートし, 上位K個の候補を保存
        candidates = sorted(tmp_candidates, key=lambda x: -x[0]/len(x[-1]))[:k]
        
        # Beam Search可視化用
        for _, _, _, _, _, y_pred in candidates:
            root.add_child([str(i) for i in y_pred])
    
    # 最もスコアの高い候補を返す
    return candidates[0][-1], root, candidates[0][-2]

In [0]:
def decode_sequence(x, max_len=100):
    u_map, h_tm1, cell_tm1 = encoder_model.predict(x)
    y_tm1 = np.array([w2i['<s>']])
    y_pred = np.array([w2i['<s>']])
    a = []
    
    while True:
        y_t, h_t, cell_t, a_t = decoder_model.predict([y_tm1, h_tm1, cell_tm1, u_map])
        h_t, cell_t = h_t[:, 0], cell_t[:, 0]
        y_t = np.argmax(y_t.flatten())
        y_pred = np.append(y_pred, [y_t])
        a.append(a_t.flatten())
        
        h_tm1, cell_tm1 = h_t, cell_t
        y_tm1 = y_pred[-1:]
        
        if y_pred[-1] == w2i['</s>'] or len(y_pred) > max_len:
            break
    return y_pred, a

In [0]:
test_x = np.array(valid_x[:1])

pred_y, root, att_a = beam_search(test_x, k=3, max_len=20)

print(' '.join([i2w[i] for i in pred_y]))

plt.imshow(test_x[0])

In [0]:
import skimage.transform

fig = plt.figure(figsize=(18, 10))

for i, (wordid, att_a_t) in enumerate(zip(pred_y[1:], att_a)):
    ax = fig.add_subplot(5, 5, i+1)
    
    # Plot image
    ax.imshow(test_x[0])
    ax.axis('off')
    
    # Plot attention
    att_a_t = skimage.transform.pyramid_expand(att_a_t.reshape(7, 7), upscale=32, sigma=20)
    ax.imshow(att_a_t, alpha=.65)
    
    # Plot word
    ax.set_title(i2w[wordid])

plt.show()

In [0]:
fig = plt.figure(figsize=(30, 10))

ax = fig.add_subplot(111, xticks=[], yticks=[])
ax.axis('off')

root.mark_best_path([str(i) for i in pred_y])

root.set_coordinates(0, 0, ax, i2w, tree_depth=root.depth)

plt.show()